In [1]:
from dotenv import load_dotenv
import os 
from alpha_vantage.timeseries import TimeSeries
import pandas as pd
import hopsworks
import re 
import modal 
#prepocessing
import requests
import pandas as pd
import json
import pandas_market_calendars as mcal
import datetime
import numpy as np
from datetime import timedelta 

In [2]:
load_dotenv()

api_key = os.environ.get('stocks_api') # Replace this with your actual API key
ts = TimeSeries(key=api_key, output_format='pandas')

# Fetch daily adjusted stock prices; adjust the symbol as needed
data, meta_data = ts.get_daily(symbol='TSLA', outputsize='full')

print(data.head())

            1. open  2. high    3. low  4. close    5. volume
date                                                         
2024-05-01   182.00   185.86  179.0100    179.99   92829719.0
2024-04-30   186.98   190.95  182.8401    183.28  127031787.0
2024-04-29   188.42   198.87  184.5400    194.05  243869678.0
2024-04-26   168.85   172.12  166.3700    168.29  109815725.0
2024-04-25   158.96   170.88  158.3600    170.18  126427521.0


In [4]:
data

,1. open,2. high,3. low,4. close,5. volume
date,,,,,
2024-05-01,182.00,185.8600,179.0100,179.99,92829719.0
2024-04-30,186.98,190.9500,182.8401,183.28,127031787.0
2024-04-29,188.42,198.8700,184.5400,194.05,243869678.0
2024-04-26,168.85,172.1200,166.3700,168.29,109815725.0
2024-04-25,158.96,170.8800,158.3600,170.18,126427521.0
...,...,...,...,...,...
2010-07-06,20.00,20.0000,15.8300,16.11,6866900.0
2010-07-02,23.00,23.1000,18.7100,19.20,5139800.0
2010-07-01,25.00,25.9200,20.2700,21.96,8218800.0


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3484 entries, 2024-05-01 to 2010-06-29
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   1. open    3484 non-null   float64
 1   2. high    3484 non-null   float64
 2   3. low     3484 non-null   float64
 3   4. close   3484 non-null   float64
 4   5. volume  3484 non-null   float64
dtypes: float64(5)
memory usage: 163.3 KB


In [6]:
meta_data

{'1. Information': 'Daily Prices (open, high, low, close) and Volumes',
 '2. Symbol': 'TSLA',
 '3. Last Refreshed': '2024-05-01',
 '4. Output Size': 'Full size',
 '5. Time Zone': 'US/Eastern'}

In [7]:
#Stock market:
def today_is_a_business_day(today):
    # Get the NYSE calendar
    cal = mcal.get_calendar('NYSE')
    schedule = cal.schedule(start_date=today, end_date=today) # Get the NYSE calendar's open and close times for the specified period
    try:
        isBusinessDay = schedule.market_open.dt.strftime('%Y-%m-%d')
        return True
    except:
        print('Today {} is not a business day'.format(today))
        return False

In [8]:
def next_business_day(today):
    
    # Real tomorrow
    tomorrow = (today + timedelta(days=1)).strftime("%Y-%m-%d")
    
    # Get the NYSE calendar
    cal = mcal.get_calendar('NYSE')

    found_next_business_day = False 
    while not found_next_business_day:
        schedule = cal.schedule(start_date=tomorrow, end_date=tomorrow) # Get the NYSE calendar's open and close times for the specified period
        try:
            isBusinessDay = schedule.market_open.dt.strftime('%Y-%m-%d') # Only need a list of dates when it's open (not open and close times)
            found_next_business_day = True
        except:
            print('The date {} is not a business day'.format(tomorrow))
            tomorrow = (datetime.datetime.strptime(tomorrow,"%Y-%m-%d") + timedelta(days=1)).strftime("%Y-%m-%d")
            
    return isBusinessDay.to_numpy()[0]

In [9]:
def extract_business_day(start_date,end_date):
    """
    Given a start_date and end_date.
    
    `Returns`:
    
    isBusinessDay: list of str (with all dates being business days)
    is_open: boolean list
        e.g is_open = [1,0,...,1] means that start_date = open, day after start_date = closed, and end_date = open
    """
    
    # Save for later
    end_date_save = end_date
    
    # Get the NYSE calendar
    cal = mcal.get_calendar('NYSE')

    # Get the NYSE calendar's open and close times for the specified period
    schedule = cal.schedule(start_date=start_date, end_date=end_date)
    
    # Only need a list of dates when it's open (not open and close times)
    isBusinessDay = np.array(schedule.market_open.dt.strftime('%Y-%m-%d')) 
    
    # Go over all days: 
    delta = datetime.timedelta(days=1)
    start_date = datetime.datetime.strptime(start_date,"%Y-%m-%d") #datetime.date(2015, 7, 16)
    end_date = datetime.datetime.strptime(end_date,"%Y-%m-%d") #datetime.date(2023, 1, 4)
    
    # Extract days from the timedelta object
    num_days = (end_date - start_date).days + 1
    
    # Create boolean array for days being open (1) and closed (0) 
    is_open = np.zeros(num_days)
    
    # iterate over range of dates
    current_BusinessDay = isBusinessDay[0]
    count_dates = 0
    next_BusinessDay = 0
    
    while (start_date <= end_date):
    
        if start_date.strftime('%Y-%m-%d') == current_BusinessDay:
            is_open[count_dates] = True

            if current_BusinessDay == end_date_save or current_BusinessDay==isBusinessDay[-1]:
                break
            else:
                next_BusinessDay += 1
                current_BusinessDay = isBusinessDay[next_BusinessDay]
        else:
            is_open[count_dates] = False

        count_dates += 1   
        start_date += delta
        
    print(np.shape(is_open))
        
    return isBusinessDay, is_open

In [10]:
def clean_column_name(name):
    # Remove all non-letter characters
    cleaned_name = re.sub(r'[^a-zA-Z]', '', name)
    return cleaned_name

In [11]:
data.columns = [clean_column_name(col) for col in data.columns]

In [12]:
data.reset_index(inplace=True)


In [13]:
data

,date,open,high,low,close,volume
0,2024-05-01,182.00,185.8600,179.0100,179.99,92829719.0
1,2024-04-30,186.98,190.9500,182.8401,183.28,127031787.0
2,2024-04-29,188.42,198.8700,184.5400,194.05,243869678.0
3,2024-04-26,168.85,172.1200,166.3700,168.29,109815725.0
4,2024-04-25,158.96,170.8800,158.3600,170.18,126427521.0
...,...,...,...,...,...,...
3479,2010-07-06,20.00,20.0000,15.8300,16.11,6866900.0
3480,2010-07-02,23.00,23.1000,18.7100,19.20,5139800.0
3481,2010-07-01,25.00,25.9200,20.2700,21.96,8218800.0
3482,2010-06-30,25.79,30.4192,23.3000,23.83,17187100.0
